In [4]:
import pandas as pd

In [5]:
# Ler os dados da base ouro já tratados e com o IOB
gold_data = pd.read_parquet("teste_token.parquet")

In [6]:
# Nos dados da base ouro, é necessário excluir entidades que não existem na base REGEX
# Também é necessário adequar a nomenclatura das entidades para o que está na base REGEX

In [7]:
deleted_entities_gold = ["CONTRATANTE", "CONTRATADA", "CONVENENTE",
                   "ORGAO_LICITANTE", "NUM_LICITACAO", "OG_ATA",
                   "IDENT_DISPENSA", "FUND_DISPENSA"]

In [8]:
keys_gold_to_regex_iob = {
    "NUM_AJUSTE": "CONTRATO",
    "PROCESSO": "PROCESSO",
    "OBJ_AJUSTE": "OBJETO",
    "DATA_ASSINATURA": "DATA_ASS.",
    "VIGENCIA": "VIGENCIA",
    "VALOR": "VALOR",
    "CODIGO_UO": "UNI_ORC.",
    "PT": "PROG_TRAB.",
    "ND": "NAT_DESP.",
    "NE": "NOTA_EMP."
}

In [9]:
deleted_entities_gold = ["I-" + l for l in deleted_entities_gold] + ["B-" + l for l in deleted_entities_gold]

In [10]:
# Removendo as entidades que não existem na base REGEX
gold_data["tokenized_text"] = gold_data["tokenized_text"].str.join("\n")

for l in deleted_entities_gold:
    gold_data["tokenized_text"] = gold_data["tokenized_text"].str.replace(l, "O", regex=False)

In [11]:
# Substituindo a nomenclatura das entidades que existem tanto na base ouro quanto na base REGEX
keys_gold_to_regex_iob_adj = dict()

for key, value in keys_gold_to_regex_iob.items():
    for b in ["B-", "I-"]:
        gold_data["tokenized_text"] = gold_data["tokenized_text"].str.replace(b+key, b+value, regex=False)

In [12]:
# Retornando ao formato original
gold_data["tokenized_text"] = gold_data["tokenized_text"].str.split("\n")

In [13]:
# Na base REGEX, é preciso deixar no formato IOB1, com labels e text juntos
# Tambémé preciso excluir as entidades que não existem na base ouro

In [14]:
# Ler os dados da base REGEX já tratados
regex_data = pd.read_csv("/home/ciri/Desktop/sofia/unb/knedle/vitor2/contratosFINAL.csv")
regex_data = regex_data.drop(columns=["Unnamed: 0"])

In [15]:
regex_data = regex_data[["text", "labels"]]

In [16]:
assert (regex_data["text"].str.split().str.len() == regex_data["labels"].str.split().str.len()).all(), "Labels e text não batem na tokenização."

In [17]:
regex_data["text"] = regex_data["text"].str.split()
regex_data["labels"] = regex_data["labels"].str.split()

In [18]:
def label_word(text, labels):
    labeled_text = []
    
    for i in range(len(text)):
        labeled_text.append(text[i] + " X X " + labels[i])
    
    return labeled_text

In [19]:
regex_data["tokenized_text"] = regex_data.apply(lambda row: label_word(row["text"], row["labels"]), axis=1)

In [20]:
deleted_entities_regex = ["PARTES", "SIGNATARIOS", "LEI_ORC."]
deleted_entities_regex = ["I-" + l for l in deleted_entities_regex] + ["B-" + l for l in deleted_entities_regex]

In [21]:
# Removendo as entidades que não existem na base REGEX
regex_data["tokenized_text"] = regex_data["tokenized_text"].str.join("\n")

for l in deleted_entities_regex:
    regex_data["tokenized_text"] = regex_data["tokenized_text"].str.replace(l, "O", regex=False)

In [22]:
regex_data["tokenized_text"] = regex_data["tokenized_text"].str.split("\n")

In [23]:
# Escrever os .txt com os dados para treinamento

In [24]:
# Para o treinamento, pegar apenas dados da base REGEX com menos de 1000 caraceteres

In [25]:
regex_data_mod = regex_data[regex_data["text"].str.join(" ").str.len() <= 1000].reset_index(drop=True)

In [31]:
regex_data_mod["treated_text"] = regex_data_mod["text"].str.join(" ")

In [36]:
regex_data_mod["base"] = "regex"
gold_data["base"] = "gold"

In [38]:
full_data = pd.concat([regex_data_mod[["treated_text", "tokenized_text", "base"]], gold_data[["treated_text", "tokenized_text", "base"]]], ignore_index=True)

In [39]:
# Salvar dados para separar amostras de teste, validação e treinamento para os códigos da Manuela
full_data.to_parquet("full_data.parquet", index=False)

In [ ]:
# Salvar dados no formato para rodar nos códigos do José Reinaldo

In [25]:
train_set = "\n\n".join(list(regex_data_mod["tokenized_text"].str.join("\n")))

In [26]:
testa_set = gold_data.sample(frac=0.5)
testb_set = gold_data[~gold_data.index.isin(testa_set.index)]

In [27]:
testa_set = "\n\n".join(list(testa_set["tokenized_text"].str.join("\n")))
testb_set = "\n\n".join(list(testb_set["tokenized_text"].str.join("\n")))

In [28]:
train_file = open("regex_train.txt", "w")
train_file.write(train_set)
train_file.close()

In [29]:
testa_file = open("regex_testa.txt", "w")
testa_file.write(testa_set)
testa_file.close()

In [30]:
testb_file = open("regex_testb.txt", "w")
testb_file.write(testb_set)
testb_file.close()